# Build Sample Dataset

## Setup

In [1]:
# Setup and Imports
import json
import pandas as pd

## Load Datasets

In [3]:
prm800k = pd.read_csv("dataset/prm800k-test-valid-data.csv")
prm800k

,labeler,timestamp,generation,is_quality_control_question,is_initial_screening_question,question,label,total_steps,total_steps_have_neg_1,steps_neg_1,total_neg_1_sequence_from_last_step,total_steps_have_zero,steps_zero,total_steps_have_pos_1_or_human,steps_pos_1_or_human,total_steps_have_combination,finish_reason
0,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-30T14:37:13.296218,NaN,False,False,{'problem': 'There are an infinite number of v...,"{'steps': [{'completions': [{'text': ""Let's se...",34,22,"[2, 3, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...",3,14,"[2, 3, 4, 6, 7, 9, 11, 13, 18, 20, 24, 26, 27,...",34,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",11,solution
1,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-30T13:26:58.414691,NaN,False,False,{'problem': 'When rolling a certain unfair six...,"{'steps': [{'completions': [{'text': ""Well, le...",35,18,"[2, 3, 4, 6, 8, 9, 12, 13, 14, 17, 18, 19, 20,...",0,11,"[3, 4, 8, 9, 12, 13, 15, 18, 21, 28, 31]",35,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",10,solution
2,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-31T14:39:30.588403,NaN,False,False,{'problem': 'Find all solutions to\n\\[\\sin \...,"{'steps': [{'completions': [{'text': ""Let's us...",34,13,"[1, 2, 6, 7, 8, 14, 16, 18, 29, 30, 31, 32, 34]",1,7,"[2, 7, 8, 14, 16, 33, 34]",34,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",6,solution
3,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-29T07:48:01.714041,NaN,False,False,{'problem': 'The solutions of the equation $z^...,{'steps': [{'completions': [{'text': 'There is...,40,17,"[2, 6, 8, 9, 11, 12, 16, 17, 18, 20, 25, 30, 3...",0,8,"[2, 3, 16, 17, 18, 20, 30, 40]",40,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",6,solution
4,e90a38f3-3135-4465-87af-3e6322e3d772,2022-07-22T20:02:50.866783,NaN,False,False,{'problem': 'A sequence $(a_n)$ is defined as ...,"{'steps': [{'completions': [{'text': ""So we're...",36,9,"[5, 9, 10, 11, 12, 13, 14, 15, 16]",0,8,"[5, 6, 9, 11, 12, 14, 16, 35]",36,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",6,solution
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,a2e09d3a-2da6-4094-89ca-e92125f6aefd,2022-11-11T23:53:32.446394,6.0,False,False,{'problem': 'Find the product of $6_8 \\cdot 7...,{'steps': [{'completions': [{'text': 'I notice...,15,0,[],0,0,[],15,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0,solution
538,2600b6ef-d904-43e8-ba92-de2fec65e275,2022-11-11T23:43:18.235408,6.0,False,False,"{'problem': 'Compute $\\dbinom{31}{28}$.', 'gr...",{'steps': [{'completions': [{'text': 'To compu...,6,1,[6],1,0,[],6,"[1, 2, 3, 4, 5, 6]",0,solution
539,fc08c519-6e30-4ca6-b223-37cc3bcb5879,2022-11-11T22:24:31.318743,6.0,False,False,"{'problem': ""Alice and Bob are playing a game....",{'steps': [{'completions': [{'text': 'I need t...,16,0,[],0,0,[],16,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0,solution
540,fc08c519-6e30-4ca6-b223-37cc3bcb5879,2022-11-11T22:19:53.088201,6.0,False,False,{'problem': 'In how many ways can $7$ people s...,{'steps': [{'completions': [{'text': 'This is ...,16,0,[],0,0,[],16,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0,solution


## Get Experiment Preparation Dataset

*The Strategy*
Idea: Disquality Not Important Datas
Collect Dataset Sample with Slicing from Big to Small

In [36]:
import json
import ast

def string_to_dict(input_string):
    input_string = input_string.strip()

    try:
        json_string = input_string.replace("'", '"')
        json_string = json_string.replace('True', 'true').replace('False', 'false').replace('None', 'null')
        return json.loads(json_string)
    except (json.JSONDecodeError, ValueError):
        pass
    
    try:
        return ast.literal_eval(input_string)
    except (ValueError, SyntaxError):
        pass
    
    try:
        if input_string.strip().startswith('{') and input_string.strip().endswith('}'):
            return eval(input_string)
    except:
        pass

    raise ValueError(f"Cannot parse string as dictionary: {input_string[:100]}...")

In [37]:
def check_rating_completion(step):
    is_neg_1 = False
    is_zero = False
    is_pos_1 = False
    neg_1 = 0
    zero = 0
    pos_1 = 0
    
    
    for idx, completion in enumerate(step['completions']):
        rating = completion['rating']
        if rating == -1 and not is_neg_1:
            is_neg_1 = True
            neg_1 += 1
        elif rating == 0 and not is_zero:
            is_zero = True
            zero += 1
        elif rating == 1 and not is_pos_1:
            is_pos_1 = True
            pos_1 += 1

    human_completion = step['human_completion']

    rating_info = {
        "-1": neg_1,
        "0": zero,
        "1": pos_1,
        "human_completion": human_completion        
    }
    
    return is_neg_1, is_zero, is_pos_1, neg_1, zero, pos_1, rating_info

In [38]:
def get_row_contains_all_rating(prm800k_train):
    results = []
    
    for index, row in prm800k_train.iterrows():
        row_neg_1 = 0
        row_zero = 0
        row_pos_1 = 0
        total_steps_have_combination = 0
        total_steps = 0

        if len(results) == 1000:
            break

        try:
            data = string_to_dict(row['label'])
            finish_reason = data['finish_reason']
            rating_check = []
            
            # Process each step
            for step_no, step in enumerate(data['steps']):
                is_neg_1, is_zero, is_pos_1, neg_1, zero, pos_1, rating_info = check_rating_completion(step)
                row_neg_1 += neg_1
                row_zero += zero
                row_pos_1 += pos_1
                total_steps += 1
                rating_check.append(rating_info)
                
                # In one steps combined from 3 category
                if is_neg_1 and is_zero and is_pos_1: 
                    total_steps_have_combination += 1

            if row_neg_1 > 0 and row_zero > 0 and row_pos_1 > 0 and total_steps_have_combination > 0 and finish_reason == "solution":
                new_result = {
                    **row.to_dict(),
                    "row_neg_1": row_neg_1,
                    "row_zero": row_zero,
                    "row_pos_1": row_pos_1,
                    "total_steps_have_combination": total_steps_have_combination,
                    "total_steps": total_steps,
                    "finish_reason": finish_reason,
                    "rating_check": rating_check,
                }
                results.append(new_result)
        
        except Exception as e:
            print(f"Error processing row {index}: {e}")
            continue
    
    # Create DataFrame from results
    results_df = pd.DataFrame(results)
    return results_df

In [39]:
df_experiment_preparation = get_row_contains_all_rating(prm800k_test)
df_experiment_preparation = df_experiment_preparation.sort_values('total_steps_have_combination', ascending=False)
df_experiment_preparation.to_csv('dataset/prm800k-experiment-preparation-test.csv', index=False)

Error processing row 100: 'NoneType' object is not iterable


In [40]:
df_experiment_preparation

,labeler,timestamp,generation,is_quality_control_question,is_initial_screening_question,question,label,row_neg_1,row_zero,row_pos_1,total_steps_have_combination,total_steps,finish_reason,rating_check
28,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-30T13:26:58.414691,NaN,False,False,{'problem': 'When rolling a certain unfair six...,"{'steps': [{'completions': [{'text': ""Well, le...",18,11,27,7,35,solution,"[{'-1': 0, '0': 0, '1': 1, 'human_completion':..."
31,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-30T14:37:13.296218,NaN,False,False,{'problem': 'There are an infinite number of v...,"{'steps': [{'completions': [{'text': ""Let's se...",22,14,19,7,34,solution,"[{'-1': 0, '0': 0, '1': 1, 'human_completion':..."
7,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-31T14:39:30.588403,NaN,False,False,{'problem': 'Find all solutions to\n\\[\\sin \...,"{'steps': [{'completions': [{'text': ""Let's us...",13,7,31,5,34,solution,"[{'-1': 1, '0': 0, '1': 1, 'human_completion':..."
41,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-08-03T08:31:13.102743,NaN,False,False,{'problem': 'If $\\arccos x + \\arccos 2x + \\...,"{'steps': [{'completions': [{'text': ""Let's us...",10,5,27,5,30,solution,"[{'-1': 0, '0': 0, '1': 1, 'human_completion':..."
10,e90a38f3-3135-4465-87af-3e6322e3d772,2022-07-22T20:02:50.866783,NaN,False,False,{'problem': 'A sequence $(a_n)$ is defined as ...,"{'steps': [{'completions': [{'text': ""So we're...",9,8,33,5,36,solution,"[{'-1': 0, '0': 0, '1': 1, 'human_completion':..."
33,e90a38f3-3135-4465-87af-3e6322e3d772,2022-07-24T20:15:48.777873,NaN,False,False,{'problem': 'In how many ways can $7$ people s...,"{'steps': [{'completions': [{'text': ""Let's st...",6,7,11,3,15,solution,"[{'-1': 0, '0': 1, '1': 1, 'human_completion':..."
45,e90a38f3-3135-4465-87af-3e6322e3d772,2022-08-05T23:00:06.860704,NaN,False,False,"{'problem': 'Let $a,$ $b,$ and $c$ be distinct...","{'steps': [{'completions': [{'text': ""Let's wr...",8,5,13,3,17,solution,"[{'-1': 0, '0': 0, '1': 1, 'human_completion':..."
40,d8aa7923-b970-45e1-9734-e4a7f6c4a7db,2022-08-02T17:50:24.697088,NaN,False,False,"{'problem': 'For every positive integer $n$, l...",{'steps': [{'completions': [{'text': 'I think ...,9,11,50,3,52,solution,"[{'-1': 0, '0': 1, '1': 1, 'human_completion':..."
5,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-08-01T16:24:53.349139,NaN,False,False,{'problem': 'Suppose that $f$ is a polynomial ...,"{'steps': [{'completions': [{'text': ""Let's re...",5,3,6,3,7,solution,"[{'-1': 1, '0': 1, '1': 1, 'human_completion':..."
42,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-08-07T10:30:43.012775,NaN,False,False,{'problem': 'What is the least positive intege...,"{'steps': [{'completions': [{'text': ""Let's wr...",9,2,26,2,28,solution,"[{'-1': 0, '0': 0, '1': 1, 'human_completion':..."


### Versioning:
1. v1.0: row_neg_1 > 0 and row_zero > 0 and row_pos_1 > 0: 37,543 rows
2. v2.0: row_neg_1 > 0 and row_zero > 0 and row_pos_1 > 0 and total_steps_have_combination > 0: 16,535 rows
3. v3.0: row_neg_1 > 1 and row_zero > 1 and row_pos_1 > 1 and total_steps_have_combination > 0: 385 rows
4. Sorting from most total_steps_have_combination
5. Collect 1000 rows
6. Collect only finish_reason == "solution"
7. Add rating check and info to check rating in every completions

## Build Fair Datasets

In [41]:
df_experiment_preparation = pd.read_csv("dataset/prm800k-experiment-preparation.csv")
df_experiment_preparation

,labeler,timestamp,generation,is_quality_control_question,is_initial_screening_question,question,label,row_neg_1,row_zero,row_pos_1,total_steps_have_combination,total_steps,finish_reason,rating_check
0,e90a38f3-3135-4465-87af-3e6322e3d772,2022-07-02T18:19:39.244785,NaN,False,False,"{'problem': 'Define the sequence $a_1, a_2, a_...",{'steps': [{'completions': [{'text': 'What exa...,34,29,48,16,64,solution,"[{'-1': 1, '0': 1, '1': 1, 'human_completion':..."
1,d8aa7923-b970-45e1-9734-e4a7f6c4a7db,2022-07-16T06:04:01.122899,NaN,False,False,"{'problem': 'Harold, Tanya, and Ulysses paint ...","{'steps': [{'completions': [{'text': ""Let's sa...",32,27,45,15,60,solution,"[{'-1': 1, '0': 1, '1': 1, 'human_completion':..."
2,e90a38f3-3135-4465-87af-3e6322e3d772,2022-07-15T16:01:50.243286,NaN,False,False,{'problem': 'What is the sum of all the positi...,"{'steps': [{'completions': [{'text': ""Let's ca...",27,22,51,12,62,solution,"[{'-1': 0, '0': 0, '1': 1, 'human_completion':..."
3,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-12T20:22:22.292498,NaN,False,False,{'problem': 'The first digit of a string of 20...,"{'steps': [{'completions': [{'text': ""Let's fi...",45,20,46,12,68,solution,"[{'-1': 0, '0': 0, '1': 1, 'human_completion':..."
4,e90a38f3-3135-4465-87af-3e6322e3d772,2022-07-07T14:48:59.553869,NaN,False,False,{'problem': 'Triangle $ABC$ has positive integ...,"{'steps': [{'completions': [{'text': ""Hey, whe...",27,25,40,12,53,solution,"[{'-1': 1, '0': 1, '1': 1, 'human_completion':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-23T17:20:48.793671,NaN,False,False,{'problem': 'What are the last two digits in t...,{'steps': [{'completions': [{'text': '1! is ju...,10,3,30,1,35,solution,"[{'-1': 0, '0': 0, '1': 1, 'human_completion':..."
425,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-27T17:24:41.556043,NaN,False,False,{'problem': 'Two transformations are applied t...,{'steps': [{'completions': [{'text': 'We can t...,3,3,14,1,16,solution,"[{'-1': 0, '0': 1, '1': 0, 'human_completion':..."
426,e90a38f3-3135-4465-87af-3e6322e3d772,2022-06-30T21:40:52.528252,NaN,False,False,{'problem': 'Solve for $x$: $x+2x = 400-(3x+4x...,{'steps': [{'completions': [{'text': 'So we ha...,1,1,7,1,7,solution,"[{'-1': 1, '0': 1, '1': 1, 'human_completion':..."
427,a5d23a92-ab95-4274-af7a-dd1315e81403,2022-07-19T15:43:48.450161,NaN,False,False,"{'problem': 'If $4^6=8^n$, what is $n$?', 'gro...",{'steps': [{'completions': [{'text': 'Since $8...,1,1,5,1,5,solution,"[{'-1': 1, '0': 1, '1': 1, 'human_completion':..."


### 1. Check Distribution